# Preparación del Entorno

## Importación de Librerías

In [1]:
# Aplicacion de modelo

# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,627623,33.6,1404608,75.1,994947,53.2
Vcells,1149754,8.8,8388608,64.0,1769393,13.5


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Loading required package: data.table

Loading required package: rpart



## Importación del archivo

In [3]:
# Aqui se debe poner la carpeta de la materia de SU computadora local
setwd("C:/Users/German/Desktop/dmeyf2023/German/Clase 4") # Establezco el Working Directory

# cargo el dataset
dataset <- fread("competencia_01_alternativo.csv")

# Definición de parámetrtos

In [4]:
# los parametros del script
#  deben copiarse a mano
PARAM <- list()

#Número de experimento
PARAM$experimento <- "KA4000-42"

# puede ser { "ternaria", "binaria1", "binaria2" }
PARAM$clase <- "ternaria"

# poner 0 si no se quiere que se haga oversampling
PARAM$oversampling <- 0

# poner algo mayor a cero si se quiere cortar por probabilidad
#  generalmente debe ser 0.025 , si se quiere cortar por probabilidad
PARAM$prob_corte <-  -1

# poner -1 si se quiere cortar por probabilidad
#  un numero mayor o igual a cero indica que se corta por cant envios
PARAM$corte <- 15000

# Cambiar acá los modelos (en este caso, uso el de la solución 18)
PARAM$rpart$cp <- -0.805955567706734
PARAM$rpart$minsplit <- 1935
PARAM$rpart$minbucket <- 394
PARAM$rpart$maxdepth <- 7

# Programa

In [5]:
if( PARAM$prob_corte <= 0 & PARAM$corte <= 0 ) stop("debe elegir al menos una opcion de corte" )
if( PARAM$prob_corte > 0 & PARAM$corte > 0 ) stop("No puede elegir dos opcinoes de corte al mismo tiempo" )
if( PARAM$clase == "binaria2" & PARAM$corte <= 0 ) stop( "en el caso de clase binaria2  debe cortar por envios, PARAM$corte" )

In [6]:
# creo la carpeta donde va el experimento
#  HT  representa  Hiperparameter Tuning
dir.create("./exp/", showWarnings = FALSE)
dir.create( paste0("./exp/", PARAM$experimento, "/"),
           showWarnings = FALSE)

# Establezco el Working Directory DEL EXPERIMENTO
setwd( paste0("./exp/", PARAM$experimento, "/") )

switch(PARAM$clase,
  "ternaria"  = dataset[, clase_nueva := ifelse( clase_ternaria=="BAJA+2", "POS", clase_ternaria ) ],
  "binaria1"  = dataset[, clase_nueva := ifelse( clase_ternaria=="BAJA+2", "POS", "NEG" ) ],
  "binaria2"  = dataset[, clase_nueva := ifelse( clase_ternaria %in% c( "BAJA+1","BAJA+2"), "POS", "NEG" ) ],
  stop(" PARAM$clase debe tener un valor valido")
)

In [7]:
dtrain <- dataset[foto_mes == 202103] # defino donde voy a entrenar

dapply <- dataset[foto_mes == 202105] # defino donde voy a aplicar el modelo
dapply[, clase_ternaria := NA ]

In [8]:
# hago el oversampling si hace falta
vector_pesos <- rep( 1.0, nrow(dtrain) )
if( PARAM$oversampling > 0 )   
  vector_pesos <- dtrain[, ifelse( clase_nueva=="POS", PARAM$oversampling, 1) ]

In [9]:
# genero el modelo,  aqui se construye el arbol
modelo <- rpart(
        formula = "clase_nueva ~ . -clase_ternaria",
        data = dtrain, # los datos donde voy a entrenar
        xval = 10,
        control = PARAM$rpart,
        weights =  vector_pesos
)

In [10]:
# aplico el modelo a los datos nuevos
prediccion <- predict(
        object = modelo,
        newdata = dapply,
        type = "prob"
)

In [11]:

# calculo en tablita lo necesario para generar el archivo prediccion
tablita <- dapply[, list(numero_de_cliente) ]
tablita[, prob := prediccion[, "POS"] ]
setorder( tablita, -prob )
tablita[, Predicted := 0L ]

if( PARAM$prob_corte > 0 ) {
  tablita[, Predicted := as.numeric(prob > PARAM$prob_corte)]
} else {
  if( PARAM$corte > 0 ) {
    tablita[ 1:PARAM$corte, Predicted := 1]
  }
}

In [12]:
# genero el submit para Kaggle
nom_archivo <- paste0( PARAM$experimento, "_kaggle.csv" )

fwrite(tablita[, list(numero_de_cliente, Predicted)],
        file = nom_archivo,
        sep = ","
)